In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plt.rc("figure", figsize=(8,4))

# Dataset

In [3]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

# fetch dataset
data_path = "../../../dataset/TSmall_sto3g/"
postfix = "TSmall_sto3g"
dim = 26
N_ELECTRONS = 30
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
#F = np.load(join(data_path, "F" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(molecules))
molecules = (molecules[:ind], molecules[ind:])

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_train, p_train, s_test, p_test = split(s_triu, p_triu, ind)

# Fetching Descriptor 

In [5]:
import pickle

descriptor = pickle.load(open("../../../models/ButadienTDescriptor/descriptor.dump", "rb"))

In [6]:
mu, std = np.load("../../../models/ButadienTDescriptor/normalisation.npy")

In [7]:
from SCFInitialGuess.utilities.dataset import StaticDataset

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=0,
    std=1
)

In [8]:
filepath = "../../../models/ButadienTDescriptor/model_descriptos_" + postfix + ".h5"
model = keras.models.load_model(filepath)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 700)               1386700   
_________________________________________________________________
dense_1 (Dense)              (None, 500)               350500    
_________________________________________________________________
dense_2 (Dense)              (None, 400)               200400    
_________________________________________________________________
dense_3 (Dense)              (None, 351)               140751    
Total params: 2,078,351
Trainable params: 2,078,351
Non-trainable params: 0
_________________________________________________________________


# Make Guess 

In [9]:
from SCFInitialGuess.utilities.dataset import AbstractDataset

G = []
for mol in molecules[1]:
    G.append(
        descriptor.calculate_all_descriptors(mol).flatten()
    )

G = np.asarray(G)
G_norm = AbstractDataset.normalize(G, mean=mu, std=std)[0]
G_norm.shape

(201, 1980)

In [10]:
p_nn = model.predict(G_norm)
p_nn.shape

(201, 351)

# Ananlysis 

In [11]:
from SCFInitialGuess.utilities.usermessages import Messenger as msg
msg.print_level = 0

In [12]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities

print(make_results_str(measure_all_quantities(
    p_nn,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=True,
    is_dataset_triu=True
)))

--- Absolute Error ---
1.04232E-03 +- 4.86030E-04
--- Symmetry Error ---
0.00000E+00 +- 0.00000E+00
--- Idempotence Error ---
1.48397E-03 +- 5.81248E-04
--- Occupance Error ---
1.72373E-02 +- 1.92167E-02
--- HF Energy Error ---
5.95613E-02 +- 6.78977E-02
--- Avg. Iterations ---
2.61990E+01 +- 2.38294E+01
--- Avg. Iterations W/O Non Converged ---
2.15132E+01 +- 1.53663E+01
--- Num. Not Convd. ---
12



In [19]:
12 / len(p_nn)

0.05970149253731343

## Damping and DIIS 

In [23]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch

iterations = np.array(measure_iterations(
    mf_initializer_damping,
    make_matrix_batch(p_nn, dim, is_triu=True),
    molecules[1]
))

print(statistics(iterations))
print(np.sum(iterations == 100))
print(statistics(iterations[iterations != 100]))

(12.616915422885572, 7.477828750626247)
0
(12.616915422885572, 7.477828750626247)


In [24]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch


iterations = np.array(list(measure_iterations(
    mf_initializer_diis,
    make_matrix_batch(p_nn, dim, is_triu=True),
    molecules[1]
)))
print(statistics(iterations))
print(np.sum(iterations == 100))
print(statistics(iterations[iterations != 100]))

(8.044776119402986, 0.9158554029057793)
0
(8.044776119402986, 0.9158554029057793)
